# Capítulo 5 - Dados ausentes

## Seção 2 - O que é um valor NaN?

No pandas, valores ausentes são representados por NaN e não são iguais a nada. Para verificar um valor ausente é necessário fazer isso chamando o método pd.isnull ou pd.notnull em vez de comparar usando == ou !=.

In [1]:
import pandas as pd
from numpy import NaN

print('NaN não é igual a nada:')
print(NaN == True)
print(NaN == False)
print(NaN == '')
print(NaN == None)
print(NaN == NaN)

print('Pra checar se é NaN preciso chamar pd.isnull(x)')
print(pd.isnull(NaN))

NaN não é igual a nada:
False
False
False
False
False
Pra checar se é NaN preciso chamar pd.isnull(x)
True


## Seção 3 - De onde vêm os valores ausentes?

Podem ser do conjunto de dados carregado ou surgido do processo de manipulação de dados (um exemplo visto no capítulo anterior foi com os merges feitos em alguns dataframes).

A função read_csv tem alguns parâmetros que permitem especificar como o dataframe será lido. O mais relevante é o parâmetro na_values que aceita uma lista de com valores possíveis de NaN.

Uma forma de se obter valores ausentes é através reindexando o dataframe original. Por exemplo:

In [2]:
gapminder = pd.read_csv('../data/gapminder.tsv', sep='\t')

life_exp = gapminder.groupby(['year'])['lifeExp'].mean()
print(life_exp)
print('---------------------------------')
print('Reindexando nos anos 2000 a 2009')
# Esse comando está no livro mas não funciona mais no pandas
# print(life_exp.loc[range(2000, 2010), ])
print(life_exp.reindex(range(2000, 2010)))

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64
---------------------------------
Reindexando nos anos 2000 a 2009
year
2000          NaN
2001          NaN
2002    65.694923
2003          NaN
2004          NaN
2005          NaN
2006          NaN
2007    67.007423
2008          NaN
2009          NaN
Name: lifeExp, dtype: float64


## Seção 4 - Trabalhando com dados ausentes

Podemos contar a quantidade de dados por coluna usando o método count de um dataframe. Com isso, subtraindo o número de linhas do retorno de count nos dá a quantidade de NaN em cada coluna:

In [3]:
ebola = pd.read_csv('../data/country_timeseries.csv')

ebola.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


In [4]:
print('Quantidade de registros presentes por coluna')
print(ebola.count())
print('-----------------------------------------------')
print('Quantidade de NaN por coluna')
print(ebola.shape[0] - ebola.count())

Quantidade de registros presentes por coluna
Date                   122
Day                    122
Cases_Guinea            93
Cases_Liberia           83
Cases_SierraLeone       87
Cases_Nigeria           38
Cases_Senegal           25
Cases_UnitedStates      18
Cases_Spain             16
Cases_Mali              12
Deaths_Guinea           92
Deaths_Liberia          81
Deaths_SierraLeone      87
Deaths_Nigeria          38
Deaths_Senegal          22
Deaths_UnitedStates     18
Deaths_Spain            16
Deaths_Mali             12
dtype: int64
-----------------------------------------------
Quantidade de NaN por coluna
Date                     0
Day                      0
Cases_Guinea            29
Cases_Liberia           39
Cases_SierraLeone       35
Cases_Nigeria           84
Cases_Senegal           97
Cases_UnitedStates     104
Cases_Spain            106
Cases_Mali             110
Deaths_Guinea           30
Deaths_Liberia          41
Deaths_SierraLeone      35
Deaths_Nigeria          84
D

Se quisermos saber a quantidade de registros NaN, podemos usar diretamente o count_nonzero do numpy:

In [5]:
import numpy as np

print(np.count_nonzero(ebola.isnull()))
print('Ou chamando o count e somando depois:')
print((ebola.shape[0] - ebola.count()).sum())

1214
Ou chamando o count e somando depois:
1214


Podemos usar o método _fillna_ do dataframe para substituir o valor de NaN por outro de nossa escolha. Por exemplo, vamos trocar NaN por 0:

In [6]:
print(ebola.fillna(0).iloc[0:10, 0:5])

         Date  Day  Cases_Guinea  Cases_Liberia  Cases_SierraLeone
0    1/5/2015  289        2776.0            0.0            10030.0
1    1/4/2015  288        2775.0            0.0             9780.0
2    1/3/2015  287        2769.0         8166.0             9722.0
3    1/2/2015  286           0.0         8157.0                0.0
4  12/31/2014  284        2730.0         8115.0             9633.0
5  12/28/2014  281        2706.0         8018.0             9446.0
6  12/27/2014  280        2695.0            0.0             9409.0
7  12/24/2014  277        2630.0         7977.0             9203.0
8  12/21/2014  273        2597.0            0.0             9004.0
9  12/20/2014  272        2571.0         7862.0             8939.0


_fillna_ pode preencher os valores NaN de acordo com o último valor anterior da série de dados (preenchimento para frente) ou com o próximo valor da série de dados (preenchimento para trás). Para isso usamos o parâmetro method='ffill' ou method='bfill'.

Note que o NaN de Cases_Guinea na linha 3 foi substituído por 2769.0, que era o valor anterior na mesma coluna. As duas primeiras posições de Cases_Liberia não foram preenchidas pois não havia valores anteriores (as posições 0 e 1 eram NaN).

In [7]:
print(ebola.fillna(method='ffill').iloc[0:10, 0:5])

         Date  Day  Cases_Guinea  Cases_Liberia  Cases_SierraLeone
0    1/5/2015  289        2776.0            NaN            10030.0
1    1/4/2015  288        2775.0            NaN             9780.0
2    1/3/2015  287        2769.0         8166.0             9722.0
3    1/2/2015  286        2769.0         8157.0             9722.0
4  12/31/2014  284        2730.0         8115.0             9633.0
5  12/28/2014  281        2706.0         8018.0             9446.0
6  12/27/2014  280        2695.0         8018.0             9409.0
7  12/24/2014  277        2630.0         7977.0             9203.0
8  12/21/2014  273        2597.0         7977.0             9004.0
9  12/20/2014  272        2571.0         7862.0             8939.0


O preenchimento para trás é parecido. Valores faltantes por último não serão preenchidos pois não há valores disponíveis para isso.

In [8]:
print(ebola.fillna(method='bfill').iloc[:, 0:5].tail())

          Date  Day  Cases_Guinea  Cases_Liberia  Cases_SierraLeone
117  3/27/2014    5         103.0            8.0                6.0
118  3/26/2014    4          86.0            NaN                NaN
119  3/25/2014    3          86.0            NaN                NaN
120  3/24/2014    2          86.0            NaN                NaN
121  3/22/2014    0          49.0            NaN                NaN


Outra forma de preencher os NaN é interpolando os dados ausentes:

In [9]:
print(ebola.interpolate().iloc[0:10, 0:5])

         Date  Day  Cases_Guinea  Cases_Liberia  Cases_SierraLeone
0    1/5/2015  289        2776.0            NaN            10030.0
1    1/4/2015  288        2775.0            NaN             9780.0
2    1/3/2015  287        2769.0         8166.0             9722.0
3    1/2/2015  286        2749.5         8157.0             9677.5
4  12/31/2014  284        2730.0         8115.0             9633.0
5  12/28/2014  281        2706.0         8018.0             9446.0
6  12/27/2014  280        2695.0         7997.5             9409.0
7  12/24/2014  277        2630.0         7977.0             9203.0
8  12/21/2014  273        2597.0         7919.5             9004.0
9  12/20/2014  272        2571.0         7862.0             8939.0


O interpolate vai deixar casos de NaN no início. Uma possibilidade é usar novamente o fillna com bfill para preencher com o próximo valor. Veja que no caso seguinte os casos na Libéria nas linhas 0 e 1 foram preenchidos com o conteúdo da linha 3:

In [10]:
print(ebola.interpolate().fillna(method='bfill').iloc[0:10, 0:5])

         Date  Day  Cases_Guinea  Cases_Liberia  Cases_SierraLeone
0    1/5/2015  289        2776.0         8166.0            10030.0
1    1/4/2015  288        2775.0         8166.0             9780.0
2    1/3/2015  287        2769.0         8166.0             9722.0
3    1/2/2015  286        2749.5         8157.0             9677.5
4  12/31/2014  284        2730.0         8115.0             9633.0
5  12/28/2014  281        2706.0         8018.0             9446.0
6  12/27/2014  280        2695.0         7997.5             9409.0
7  12/24/2014  277        2630.0         7977.0             9203.0
8  12/21/2014  273        2597.0         7919.5             9004.0
9  12/20/2014  272        2571.0         7862.0             8939.0


Observações podem ser descartadas com dropna, que pode ser configurado para descartar os registros se uma linha inteira (how='all') ou apenas parte dela (how='any' combinado com thresh=min_valores_validos) tiver NaN.

Para finalizar, o cálculo envolvendo valores NaN também retorna NaN. É possível informar para descartar esses valores do cálculo:

In [11]:
print(ebola.Cases_Guinea.sum(skipna=True))
print(ebola.Cases_Guinea.sum(skipna=False))

84729.0
nan
